In [ ]:
import pandas as pd

from modules.module_for_imports import *

# Game_events analysis

In [ ]:
game_events = get_game_events('../')
game_events.info()

In [ ]:
game_events['type'].value_counts(dropna=False)

In [ ]:
game_events.query('player_in_id.isnull()').count()

In [ ]:
game_events['game_id'].value_counts().head(40)

In [ ]:
game_events.query('type == "Substitutions"')['description'].value_counts()

In [ ]:
game_events.query('type == "Shootout"')['description'].value_counts()

In [ ]:
has_assist = game_events['description'].astype(str).str.contains("Assist")
goals_without_assist = game_events[~has_assist]
goals_without_assist.query('type == "Goals"').count()

In [ ]:
game_events['type'] = game_events['type'].astype('string') 
game_events['date'] = pd.to_datetime(game_events['date'].astype('string'))
game_events.fillna({'player_in_id':-1, 'player_assist_id':-1}, inplace=True)
game_events['player_in_id'] = game_events['player_in_id'].astype('int')
game_events['player_assist_id'] = game_events['player_assist_id'].astype('int')
game_events.info()

In [ ]:
game_events.head(13)

> # In description "2." is the second time someone does something:
> There are different types of events:
> - cards: type of card: ?(cause of card)
> - goals: 
>   how goal has been scored: number of the goal in the specified context, 
>   **if there is an assist**: how it has been done, number of the assist by that player in the specified context
> - substitution: specified or not
> - shootout: (shootouts are like 1v1 for 5" and have been alternatives to the penalties(english for 'rigori'))

In [ ]:
game_events['player_assist_id'].isna().sum()

In [ ]:
game_events.info()

In [ ]:
game_events['player_in_id'].value_counts().sum()

In [ ]:
game_events['minute'].value_counts()

In [ ]:
game_events.query('description.astype("string").str.contains("Not reported")', engine='python').value_counts('description')

In [ ]:
game_events['type'].value_counts()

> # It seems that only substitution type has "not reported" value

> ## Formatting all type tuples

In [ ]:
game_events['description'].replace([float('NaN'), ', Not reported'], None).value_counts(dropna=False)

In [ ]:
# description cleaning
# removing NaNs and ', Not reported' values
game_events['description'].replace([float('NaN'), ', Not reported'], None, inplace=True)
#Substitutions is the only type that has None value on description column now

def format_description(description_value = ''):
    if description_value is not None:
        description_list = description_value.split(', ')
        return [format_string(des_elem) for des_elem in description_list]
    else: 
        return description_value
    
def format_string(des_elem = ''):
    # pre: des_elem is not None
    if des_elem[0].isdigit():
            substr = des_elem.split(".")
            if substr[0][0] == '1':
                substr[0] = substr[0] + 'st'
            elif substr[0][0] == '2':
                substr[0] = substr[0] + 'nd'
            elif substr[0][0] == '2':
                substr[0] = substr[0] + 'rd'
            else:
                substr[0] = substr[0] + 'th'      
            return ''.join(substr)
    return des_elem

# modifying all type tuples
target = game_events['description'].apply(lambda x: x if x is None else x[2:] if x[0]==',' else x)
target = target.apply(format_description)


# @Todo: move all this in Assignment_datasets.py
target.head()

In [ ]:
game_events['description'].update(target)
target = None
game_events.query('type=="Substitutions"')['description'].value_counts(dropna=False)